In [1]:
import os
import sys
import json
import re

from glob import glob
from logging import error, warning

from common import CLASS_ABBREV_MAP, FINNISH_MAIN_REGISTER, load_conllu


CLASS_COMMENT_RE = re.compile(r'^#.*?\bregister:(.*)')


def get_class_from_comments(comments):
    class_ = None
    for comment in comments:
        m = CLASS_COMMENT_RE.match(comment)
        if m:
            if class_:
                raise ValueError('duplicate class')
            class_ = m.group(1)
    return class_


def load_conllu_with_class(fn):
    sentences, class_ = [], None
    for comments, sentence in load_conllu(fn):
        c = get_class_from_comments(comments)
        if c is not None:
            if class_ is not None:
                raise ValueError('duplicate class')
            class_ = c
        sentences.append(sentence)
    if class_ is None:
        raise ValueError('missing class in {}'.format(fn))
    class_ = FINNISH_MAIN_REGISTER[class_]
    return sentences, CLASS_ABBREV_MAP[class_]


def load_parsed_data(dirpath):
    parses, classes = [], []
    for fn in glob('{}/*.conllu'.format(dirpath)):
        sentences, class_ = load_conllu_with_class(fn)
        if class_ is None:
            continue    # class doesn't map across languages
        parses.append(sentences)
        classes.append(class_)
    return parses, classes


train_parses, train_classes = load_parsed_data('../data/split-parsed/train/')
devel_parses, devel_classes = load_parsed_data('../data/split-parsed/dev/')

In [2]:
from collections import Counter


MIN_EXAMPLES = 25    # filter classes with fewer


class_count = Counter()
for c in train_classes:
    class_count[c] += 1
target_class = set(c for c, v in class_count.items() if v >= MIN_EXAMPLES)


def filter_by_class(parses, classes, targets):
    filtered_parses, filtered_classes = [], []
    for t, c in zip(parses, classes):
        if c in targets:
            filtered_parses.append(t)
            filtered_classes.append(c)
    return filtered_parses, filtered_classes


train_parses, train_classes = filter_by_class(train_parses, train_classes, target_class)
devel_parses, devel_classes = filter_by_class(devel_parses, devel_classes, target_class)

In [3]:
from pprint import pprint

def class_counts(classes):
    counter = Counter()
    for c in classes:
        counter[c] += 1
    return counter


pprint(class_counts(train_classes))

Counter({'Narrative': 422,
         'Info-Persuasion': 269,
         'D-Informational': 218,
         'Opinion': 199,
         'How-to': 86,
         'Discussion': 84})


In [16]:
def words_from_sentences(sentences):
    words = []
    for sentence in sentences:
        for word in sentence:
            words.append(word.form)
    return words


def texts_from_parses(parses):
    texts = []
    for sentences in parses:
        texts.append(' '.join(words_from_sentences(sentences)))
    return texts


train_texts = texts_from_parses(train_parses)
devel_texts = texts_from_parses(devel_parses)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC


vectorizer = TfidfVectorizer(analyzer='word', lowercase=False, ngram_range=(1,3))
vectorizer.fit(train_texts)

train_X = vectorizer.transform(train_texts)
devel_X = vectorizer.transform(devel_texts)

classifier = LinearSVC(C=1.0)
classifier.fit(train_X, train_classes)

classifier.score(devel_X, devel_classes)

0.5966850828729282

In [18]:
import eli5


eli5.show_weights(classifier, vec=vectorizer, top=(100,100))

In [20]:
import numpy as np

from sklearn.metrics import confusion_matrix
from pandas import DataFrame


pred_Y = classifier.predict(devel_X)
cm = confusion_matrix(devel_classes, pred_Y)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]    # normalize
df = DataFrame(cm * 100, index=classifier.classes_, columns=classifier.classes_)
df.round(2)

,D-Informational,Discussion,How-to,Info-Persuasion,Narrative,Opinion
D-Informational,16.13,0.0,0.00,29.03,38.71,16.13
Discussion,0.00,0.0,0.00,0.00,81.82,18.18
How-to,25.00,0.0,16.67,25.00,25.00,8.33
Info-Persuasion,2.70,0.0,0.00,72.97,16.22,8.11
Narrative,0.00,0.0,0.00,6.56,81.97,11.48
Opinion,0.00,0.0,0.00,3.45,13.79,82.76
